In [ ]:
!pip install 

In [ ]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import cv2
import matplotlib.pyplot as plt

In [ ]:
#Definindo diretórios;

dir_geral = os.path.abspath("Projeto Final.ipynb")
dir_geral = os.path.dirname(dir_geral)
dir_csv = os.path.join(dir_geral + "\\Medical mask\\train.csv")
dir_img = os.path.join(dir_geral + "\\Medical mask\\images\\")
dir_json = os.path.join(dir_geral+"\\Medical mask\\meta.json")
print("Diretório geral: {} \nDiretório CSV: {} \nDiretório Imagem: {} \nDiretório metadata: {}\n".format(dir_geral, dir_csv, dir_img, dir_json))

Diretório geral: /content 
Diretório CSV: /content\Medical mask\train.csv 
Diretório Imagem: /content\Medical mask\images\ 
Diretório metadata: /content\Medical mask\meta.json



In [ ]:
###### PRÉ-PROCESSAMENTO DATASET-LABELS ######
#Lendo a base com o nome das fotos e a classificação;

df = pd.read_csv(dir_csv)
df.head()

In [ ]:
#Excluindo linhas duplicadas, com NaN e ordenando o DataFrame pelo nome;

df = df.drop_duplicates(subset = ['name'])
df = df.dropna()
df = df.sort_values(by = 'name')
index = list(range(0,(len(df.index))))
df['index'] = index
df.reset_index(inplace = True)
df.rename(columns = {'name':'id'}, inplace = True)
df.set_index('index', inplace = True)
df = df.drop(columns = ['level_0'])
print(df)

#df = df.reindex(index)
#df = df.groupby(['classname'], as_index = False).sum()
#print(df)

In [ ]:
#Identificando categorias definem as imagens de quem usa máscara ('face_with_mask', 
#'mask_colorful' e 'mask_surgical') e renomeando-as em apenas duas: 'mask' e 'no_mask';

df_remove = df.loc[(df['classname'] != 'face_no_mask') & (df['classname'] != 'face_with_mask') & (df['classname'] != 'mask_colorful') & (df['classname'] != 'mask_surgical')]
df = df.drop(df_remove.index)

labels = ['mask' if ((classes == 'face_with_mask') or (classes == 'mask_colorful') or (classes == 'mask_surgical')) else 'no_mask' for classes in df['classname']]
df['classname'] = labels
df = df.rename(columns = {'classname': 'classification'}, index = {'name':'index'})

print(df)

In [ ]:
#É preciso conferir se o dataset está balanceado, pra que o treinamento não seja tendencioso

count_mask = 0

for name in df['classification']:
    
    if name == 'mask':
        count_mask += 1
print('Tem-se: \n\n{} imagens de pessoas com máscaras; \n{} imagens de pessoas sem máscara; \n{} imagens ao todo;'.format(count_mask, (len(labels) - count_mask), len(labels)))

In [ ]:
#Mais de 50% da nossa base é de pessoas com máscaras, o que pode ocasionar uma predição tendenciosa
#para pessoas com máscara - que é o oposto do que se deseja aqui. Apesar disso, caso se tire essa 
#quantidade de dados, a perda vai ser muito grande.

i = 0
for name in df['classification']:
    if name == 'mask':
        count_mask += 1
        if count_mask > 1929:
            loc = i
    else:
        continue

filter_balance = df[df.index > 1929]
filter_balance = filter_balance[filter_balance['classification']=='mask']
print(filter_balance.index.values)

df = df.drop(index = filter_balance.index.values)
#train_labels = df['classification']
#print(train_labels)

#df = df.drop(remove_25.index, axis = 0)

In [ ]:
#O arquivo 'Annotations' contém uma relação dos rostos presentes nas imagens e suas localizações 
#em pixels que pode ser usado como 'label' para conferir se o classificador está conseguindo 
#reconhecer a localização das faces:

#Lendo o JSON 'Annotations' e realizar a seleção d

import json


def get_json(name):
    if name.split('.')[-1]!='json':
        name = name + '.json'
    with open(os.path.join('Medical mask/annotations', name), 'r') as f:
        return json.load(f)
    
def get_bounding_box(name):
    
    for j in get_json(name).get("Annotations"):
        x, y, w, h = j["BoundingBox"]
        return x, y, w, h

def get_label(name):
    for j in get_json(name).get("Annotations"):
        label = j["isProtected"]
        return label

print(get_bounding_box('1801.jpg'))


In [ ]:
#Relacionando o nome das imagens ao JSON de metadados das imagens, para pegar a localização 
#de cada imagem. Assim, é possível relacioná-la à sua classificação;
###### PRÉ-PROCESSAMENTO DATASET-IMAGENS (DADOS) ######

#Análise das imagens e validação dos dados com as informações adquiridas do dataset

#conferindo os nomes das imagens e quais estão presentes no diretório de imagens
notes = os.listdir(dir_geral+'\\Medical mask\\annotations')
imgs_names = os.listdir(dir_img)
print(len(notes))
print(len(imgs_names))

#conferindo se as 1800 primeiras imagens são mesmo 1800
count = -1
for img in imgs_names:
    count += 1
    img = img.split('.')[0]
    if img == '1800':
        final = count
        break
        
#separando set de validação de imagem dos demais
validation_imgs = imgs_names[:final]
imgs_names = imgs_names[final+1:]

#conferindo se há uma imagem para cada label do csv
img_w_data = []
for ids in df['id']:
    ids = ids.split('.')[0]
    for imgs in imgs_names:
        img = imgs
        imgs = imgs.split('.')[0]
        if imgs == ids:
            img_w_data.append(img)
            break

#TOTAL FILES DF 2150

In [ ]:
largura = []
altura = []

for i in img_w_data:
    x, y, w, h = get_bounding_box(i)
    largura.append(w - x)
    altura.append(h - y)

# calcula o valor médio das medidas de interesse.
media_largura, media_altura = np.mean(largura), np.mean(altura)

In [ ]:
#Pré-processamento das imagens, recuperação da região de interesse e automação do 
#reconhecimento de faces


#função que permite a recuperação mais fácil da imagem
def creat_path(name):
    path = os.path.join('Medical mask/images', name)
    return path

def padronizar_imgs(name):
    imagem = cv2.imread(creat_path(name), cv2.IMREAD_GRAYSCALE)
    imagem = cv2.resize(imagem, (300, 300))
    return imagem

#testando funções
#ex1 = padronizar_imgs(img_w_data[0])
#plt.imshow(ex1, cmap ='gray')

In [ ]:
for i in range(len(img_w_data)):
    for id in df['id']:
        if i == id:
            x1, x2, y1, y2 = df.iloc[id]['x1','x2','y1','y2']

In [ ]:
bbs = []
for i in range(len(df)):
    data = []
    for j in df.iloc[i]:
        data.append(j)
    bbs.append(data)
    
df['boxes'] = bbs

import random

data = []
labels = {'mask':0, 'no_mask':1}
for i in df["id"]:
    lbl = get_label(i)
    x, y, w, h = get_bounding_box(i)
    img = padronizar_imgs(i)
    if lbl == True:
        data.append([img, labels['mask']])
    if lbl == False:
        data.append([img, labels['no_mask']])
random.shuffle(data)

In [ ]:
x = []
y = []
for features, labels in data:
    x.append(features)
    y.append(labels)

x_train = x
y_train = np.array(y)
data[0][0]
#plt.imshow(data[0][0])

In [ ]:
from keras.models import Sequential
from keras import optimizers
from keras import backend as K
from keras.layers import LSTM
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator

x = np.array(x)
x = x.reshape(-1, 300, 300, 1)
y = np.array(y)

model = keras.Sequential()
# Add an Embedding layer expecting input vocab of size 1000, and
# output embedding dimension of size 64.
model.add(layers.Embedding(input_dim=1000, output_dim=64))

# Add a LSTM layer with 128 internal units.
model.add(layers.LSTM(128))

# Add a Dense layer with 10 units.
model.add(layers.Dense(10))

model.summary()
model.fit(x, y, epochs = 10, batch_size = 5)


In [ ]:
from mtcnn import MTCNN

detector = MTCNN()
name_img = test_images[0]
img = get_json(name_img)
face = detector.detect_faces(img)
for face in face:
        bounding_box=face['box']
        x=cv2.rectangle(img,
              (bounding_box[0], bounding_box[1]),
              (bounding_box[0]+bounding_box[2], bounding_box[1] + bounding_box[3]),
              (0,155,255),
              10)
        plt.imshow(x)

In [ ]:
def draw_face(filename, result_list):
  data = plt.imread(filename)
  plt.imshow(data)
  for result in result_list:
    x, y, width, height = result['box']
    # create the shape
    rect = plt.Rectangle((x, y), width, height, fill=False, color='green')
    # draw the box
    ax.add_patch(rect)
    # show the plot
    plt.show()


# filename = 'test1.jpg' # filename is defined above, otherwise uncomment
# load image from file
# pixels = plt.imread(filename) # defined above, otherwise uncomment
# detector is defined above, otherwise uncomment
#detector = mtcnn.MTCNN()
# detect faces in the image
faces = detector.detect_faces(pixels)
# display faces on the original image
draw_facebox(filename, faces)
